# Homework 1


The homework consists of two parts: theoretical part (10 pts) and coding part (20 pts).
 - All theoretical questions must be answered in your own words, do not copy-paste text from the internet. Points can be deducted for terrible formatting or incomprehensible English.
 - Code must be commented. If you use code you found online, you have to add the link to the source you used. There is no penalty for using outside sources as long as you convince us you understand the code.

*Once completed zip the entire directory containing this exercise and upload it to https://courses.cs.ut.ee/2020/nn/spring/Main/Practices.*

For background reading see http://cs231n.github.io/classification/.

If you did this homework together with course mates, please write here their names (answers still have to be your own!).

**Name(s):** *fill this in if applicable*

## Part 1: Lecture Materials (10 pts)

These questions are about the material covered in the two lectures about "Probability and Information theory" and "Basics of Machine Learning".

### Probability theory

**Task 1.1 (3pts)**
Some guys made a study to find out if intake of vitamin C helps you recover faster from a common cold. They asked 1000 people about their recent illness and if they consumed additional vitamin C. The results are summarized in table below:

|                       -            | Recovered in <3 days | Recovered in 3-5 days | Recovered in >5 days | Total probability of dose |
|------------------------------------|----------------------|-----------------------|----------------------|---------------------------|
| **No vitamin C**                       | 200 participants     | 150 participants      | 150 participants     |                           |
| **Low dose of vitamin C**              | 80 participants      | 60 participants       | 60 participants      |                           |
| **High dose of vitamin C**             | 120 participants     | 90 participants       | 90 participants      |                           |
| **Total probability of recovery time** |         -            |        -              |         -            |

Answer the following questions, also include explanation or calculations that led to this answer. (It is probably best if you turn the counts into probabilities first and calculate marginal probabilities)
 1. What proportion of the participants took more than 5 days to recover from illness?
 2. What is the probability to recover in less than 3 days given that you took no vitamin C? (HINT: it is a conditional probability)
 3. What is the probability to recover in less than 3 days given that you took high dose of vitamin C?
 4. Are the two variables (dose and recovery time) independent? Explain your answer.

**Your Answer**: *fill this in.*

**Task 1.2 (3pts)** Imagine we have a classifier that classifies an image into one of 4 categories: cars, bicycles, motorbikes and pedestrians. Given an image of a bicycle, it produces the following probabilities for the 4 classes: car 0.05, bicycle 0.70, motorbike 0.20, pedestrian 0.05. Considering that the true probability distribution is $[0,1,0,0]$, find (use base 2 for all the logarithms):
 - The entropy of the true distribution, the predicted distribution and a uniform distribution ($[0.25,0.25,0.25,0.25]$). Which one is the highest?
 - Kullback-Leibler divergence between the true and the predicted distribution.<br/> (notice that $KL(p||q)$ is not equal to $KL(q||p)$ )
 - Sum up the entropy of the true distribution and the KL divergence calculated above to get the cross-entropy value. Calculate the cross-entropy also using the formula $CE(p,q) = - \sum_i p_i log_2(q_i).$

**Your Answer**: *fill this in.*

### Machine Learning

**Task 1.3 (1pt)**
Suppose you have measured a series of pairs of values $y_i$ and $x_i$, for $i=1,\dots,N$, and you would like to establish some relation between the X and Y variables. Assume also that each sample is independent and identically distributed. Show that the maximum likelihood estimator for the Gaussian model of linear regression corresponds to minimizing the mean squared error between the prediction and the true output. Gaussian model assumes that the probability of some target value y given x is the Gaussian distribution $\mathcal{N}(w\cdot x, \sigma^2)$, where w are the weights.

In particular demonstrate that maximizing
$
%\arg \max_\theta 
\log \left(\frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(y - \hat{y})^2}{2\sigma^2}}\right)
$
is equivalent to minimizing 
$
%= \arg \min_\theta 
(y - \hat{y})^2
$, where $y$ is the target value, $\hat{y}=w\cdot x$ is the predicted value and variance $\sigma^2$ is considered constant.

**Your Answer**: *fill this in.*

**Task 1.4 (1pt)** Imagine you have trained a very powerful neural network to classify if a given image has a dog or a cat on it. The network achieves 99.9% accuracy on the images that it was trained on, but has only 65% accuracy on cat/dog images it has never seen before (test images). How is this problem called? How to make the difference between training and testing error smaller (name at least one solution)?

**Your Answer**: *fill this in.*

**Task 1.5: Curse of Dimensionality (2pts)**
Images are very common data type in deep learning. Each colored image consists of pixels, each pixel is a combination of three color values - red,green,blue - that have 256 possible values each.
 - How many different colors can one pixel take?
 - How many different $2\times2$ pixel images are there?
 - In the ImageNet dataset the pictures are of size $256\times256$ pixels. Compare the number of possible images with the number of images in the dataset (1.2 million).

**Your Answer**: *fill this in.*

## Part 2: k-Nearest Neighbor (kNN) exercise (20 pts)

In this assignment you will
 - understand the basic Image Classification pipeline and the data-driven approach (train/predict stages),
 - understand the train/val/test splits and the use of validation data for hyperparameter tuning,
 - develop proficiency in writing efficient vectorized code with numpy,
 - implement and apply a k-Nearest Neighbor (kNN) classifier.

### Requirements

All the required packages should be part of Anaconda already, the only additional install is `future` package (only needed if using Python 2) :
```
conda install future
```

### Downloading data

Download the CIFAR-10 dataset from http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and untar it to <br/> `../datasets`:
```
tar -xzvf cifar-10-python.tar.gz
```

### kNN

The kNN classifier consists of two stages:

- Stage I: During training, the classifier takes the training data and simply remembers it
- Stage II: During testing, kNN classifies every test image by comparing it to all training images. The most common label among the k most similar training examples is given as the output.

The optimal value of k is found via cross-validation

In [ ]:
# Run some setup code for this notebook.

from __future__ import print_function

import random
import numpy as np
from data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline

# Other plotting-related parameters
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules.
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Load the raw CIFAR-10 data.
cifar10_dir = '../datasets/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Subsample the data for more efficient code execution in this exercise
# We use just 1/10 th of the data
num_training = 5000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 500
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

In [ ]:
# Reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

In [ ]:
from k_nearest_neighbor import KNearestNeighbor

# Create a kNN classifier instance. 
# Remember that training a kNN classifier is a noop: 
# the Classifier simply remembers the data and does no further processing 
classifier = KNearestNeighbor()
classifier.train(X_train, y_train)

We would now like to classify the test data with the kNN classifier. Recall that we can break down this process into two steps: 

1. Phase I of testing: First we must compute the distances between all test examples and all train examples. 
2. Phase II of testing: Given these distances, for each test example we find the k nearest examples and have them vote for the label

Lets begin with computing the distance matrix between all training and test examples. For example, if there are **N_tr** training examples and **N_te** test examples, this stage should result in a **N_te x N_tr** matrix where each element $(i,j)$ is the distance between the $i$-th test and $j$-th train example.

**Task 2.1 (3pts):** First, open `k_nearest_neighbor.py` and implement the function `compute_distances_two_loops` that uses a (very inefficient) double loop over all pairs of (test, train) examples and computes the distance matrix one element at a time.

In [ ]:
# TODO
# Open k_nearest_neighbor.py and implement
# compute_distances_two_loops.

# Test your implementation:
dists = classifier.compute_distances_two_loops(X_test)
print("The shape of the distances matrix:", dists.shape)

assert not np.all(dists==0), "ERROR: The function seems to not be implemented, the output is all zeros"
assert np.isclose(dists[0,1],4210.59603857), "ERROR: The function seems to not be correct - an answer does not match with what we got"

In [ ]:
# We can visualize the distance matrix: each row reflects a single test example and
# its distances to training examples
plt.imshow(dists, interpolation='none')
plt.xlabel("train images")
plt.ylabel("test images")
plt.show()

**Task 2.2 (1pt):** Notice the structured patterns in the distance matrix, where some rows or columns are visible brighter. (Note that with the default color scheme black indicates low distances while white indicates high distances.)

- What in the data is the cause behind the distinctly bright rows?
- What causes the columns?

**Your Answer**: *fill this in.*



**Task 2.3 (3pts):** Now implement the function `predict_labels` in `k_nearest_neighbor.py` and run the code below:

In [ ]:
# We use k = 1 (which is Nearest Neighbor).
y_test_pred = classifier.predict_labels(dists, k=1)

# Compute and print the fraction of correctly predicted examples
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))
assert np.isclose(accuracy,0.274), "The accuracy is not correct, there is some problem"

You should expect to see approximately `27%` accuracy. Now lets try out a larger `k`, say `k = 5`:

In [ ]:
y_test_pred = classifier.predict_labels(dists, k=5)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))
assert np.isclose(accuracy,0.278), "The accuracy is not correct, there is some problem"

You should expect to see a slightly better performance than with `k = 1`.

**Task 2.4 (3pts):** Now lets speed up distance matrix computation by using partial vectorization with one loop. Implement the function `compute_distances_one_loop` in `k_nearest_neighbor.py` and run the code below:

In [ ]:
dists_one = classifier.compute_distances_one_loop(X_test)

# To ensure that our vectorized implementation is correct, we make sure that it
# agrees with the naive implementation. There are many ways to decide whether
# two matrices are similar; one of the simplest is the Frobenius norm. In case
# you haven't seen it before, the Frobenius norm of two matrices is the square
# root of the squared sum of differences of all elements; in other words, reshape
# the matrices into vectors and compute the Euclidean distance between them.
difference = np.linalg.norm(dists - dists_one, ord='fro')
print('Difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

**Task 2.5 (6pts):** Now implement the fully vectorized version inside `compute_distances_no_loops` in `k_nearest_neighbor.py` and run the code

In [ ]:
dists_two = classifier.compute_distances_no_loops(X_test)

# check that the distance matrix agrees with the one we computed before:
difference = np.linalg.norm(dists - dists_two, ord='fro')
print('Difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [ ]:
# Let's compare how fast the implementations are
def time_function(f, *args):
    """
    Call a function f with args and return the time (in seconds) that it took to execute.
    """
    import time
    tic = time.time()
    f(*args)
    toc = time.time()
    return toc - tic

two_loop_time = time_function(classifier.compute_distances_two_loops, X_test)
print('Two loop version took %f seconds' % two_loop_time)

one_loop_time = time_function(classifier.compute_distances_one_loop, X_test)
print('One loop version took %f seconds' % one_loop_time)

no_loop_time = time_function(classifier.compute_distances_no_loops, X_test)
print('No loop version took %f seconds' % no_loop_time)

# you should see significantly faster performance with the fully vectorized implementation

### Cross-validation

We have implemented the k-Nearest Neighbor classifier but we set the value `k = 5` arbitrarily. We will now determine the best value of this hyperparameter with cross-validation.

**Task 2.6 (1pt):**

In [ ]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []
################################################################################
# TODO:                                                                        #
# Split up the training data into folds. After splitting, X_train_folds and    #
# y_train_folds should each be lists of length num_folds, where                #
# y_train_folds[i] is the label vector for the points in X_train_folds[i].     #
# Hint: Look up the numpy.array_split function.                                #
################################################################################
pass
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################
assert np.shape(X_train_folds)==(5, 1000, 3072), "the shape of folds is not correct"

**Task 2.7 (3pts):**

In [ ]:
# A dictionary holding the accuracies for different values of k that we find
# when running cross-validation. After running cross-validation,
# k_to_accuracies[k] should be a list of length num_folds giving the different
# accuracy values that we found when using that value of k.
k_to_accuracies = {}


################################################################################
# TODO:                                                                        #
# Perform k-fold cross validation to find the best value of k. For each        #
# possible value of k, run the k-nearest-neighbor algorithm num_folds times,   #
# where in each case you use all but one of the folds as training data and the #
# last fold as a validation set. Store the accuracies for all fold and all     #
# values of k in the k_to_accuracies dictionary.                               #
################################################################################
pass
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

# Print out the computed accuracies
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

In [ ]:
# plot the raw observations
for k in k_choices:
    accuracies = k_to_accuracies[k]
    plt.scatter([k] * len(accuracies), accuracies)

# plot the trend line with error bars that correspond to standard deviation
accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
plt.errorbar(k_choices, accuracies_mean, yerr=accuracies_std)
plt.title('Cross-validation on k')
plt.xlabel('k')
plt.ylabel('Cross-validation accuracy')
plt.show()

print('Best k:', k_choices[np.argmax(accuracies_mean)])

In [ ]:
# Based on the cross-validation results above, choose the best value for k,   
# retrain the classifier using all the training data, and test it on the test
# data. You should be able to get above 28% accuracy on the test data.

#TODO: fill this in with best value
best_k = 1

classifier = KNearestNeighbor()
classifier.train(X_train, y_train)
y_test_pred = classifier.predict(X_test, k=best_k)

# Compute and display the accuracy
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))